# Programa para extrapolar las velocidades a partir de las condiciones de orografía y uso de suelo

### Paqueterías utilizadas 

In [ ]:
import xarray as xr 
import iris  
import pandas as pd
import numpy as np
import datetime 
import missingno as msno
import iris.pandas
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.cm as c
import time
from metpy.cbook import get_test_data
from dask.distributed import Client
#import wradlib as wrl
from scipy import interpolate
import scipy
from sklearn.metrics import mean_squared_error
from math import sqrt

# Coordenadas originales (cada 1km)

In [ ]:
lat_e1km = np.linspace(16.224533, 16.134575,11)
lon_e1km = np.linspace(-93.94673,-93.85307 , 11)

In [ ]:
coordenadas1 = []
for i in range(0,(len(lat_e1km))):
    for j in range(0, len(lon_e1km)):
        coordenadas1.append([lon_e1km[j], lat_e1km[i]])
dfcoor_Or = pd.DataFrame(coordenadas1, columns = ['Longitud', 'Latitud'])

# Coordenadas 100m

In [ ]:
lat_10km_ev100m = np.linspace(16.224533, 16.134575,101)
lon_10km_ev100m = np.linspace(-93.94673,-93.85307 , 101)

In [ ]:
coordenadas = []
for i in range(0,(len(lat_10km_ev100m))):
    for j in range(0, len(lon_10km_ev100m)):
        coordenadas.append([lon_10km_ev100m[j], lat_10km_ev100m[i]])
dfcoor = pd.DataFrame(coordenadas, columns = ['Longitud', 'Latitud'])

# Archivo Altitudes (DEM NASA)

Lectura de archios DEM sobre la altitud del terreno

In [ ]:
dfAltitud   = pd.read_csv('../Orografia_rugo/caso_D/Alt_D.csv')
dfAltitud_1 = pd.read_csv('../Orografia_rugo/caso_D/Alt_D.csv')

# Archivo Uso de Suelo (INEGI)

Lectura del archivo de uso de suelo de INEGI

In [ ]:
dfRug_INEGI = pd.read_csv('../Orografia_rugo/caso_D/Rug_D.csv') 

### Asignación de longitud de rugosidad a cada uso de suelo ([1] Manwell)

In [ ]:
rugoMan = []
for i in range(len(dfRug_INEGI)):
    z0 = dfRug_INEGI.Indice[i]
    if z0 == 1: #ATAP
        ValR1 = 50e-3
    if z0 == 2: #AH
        ValR1 = 250e-3
    if z0 == 3: #PC
        ValR1 = 10e-3
    if z0 == 4: #PI
        ValR1 =  10e-3
    if z0 == 5: #S
        ValR1 = 100e-3
    if z0 == 6: #SDE 
        ValR1 = 100e-3
    if z0 == 7: #SG
        ValR1 = 500e-3
    if z0 == 8: #VSASG
        ValR1 = 100e-3
    if z0 == 9: #VSASBC
        ValR1 = 250e-3
    if z0 == 10: #VSASMSC
        ValR1 = 250e-3
    if z0 == 11: #VSHSBC
        ValR1 = 250e-3
    if z0 == 12: #MSC
        ValR1 = 10e-3
    if z0 == 13: #MS
        ValR1 =  50e-3
    if z0 == 14: #ARA
        ValR1 = 50e-3
    if z0 == 15: #MX
        ValR1 = 10e-3
    if z0 == 16: #S/V
        ValR1 = 8e-3
    if z0 == 17: #ARAS
        ValR1 = 50e-3
    if z0 == 18: #ATA
        ValR1 =50e-3
    if z0 == 19: #SBC
        ValR1 = 500e-3
    if z0 == 20: #VSASBEC
        ValR1 = 250e-3
    if z0 == 21: #CA
        ValR1 = 0.2e-3
    if z0 == 22: #VSAMET
        ValR1 = 100.00e-3
    rugoMan.append(ValR1)
rugoMan = pd.DataFrame(rugoMan)

In [ ]:
dfRug_INEGI['zo_1'] = rugoMan

## Estáticos para el aumento de la resolución

### Orografía de la zona

In [ ]:
alts_vect = []
k = 0
for i in range (0, 101):
    arralts_vect = []
    for j in range(0,101):
        arralts_vect.append(dfAltitud.Altitudes[k])
        k=k+1
    #Umatrix = np.append(Umatrix, [arrU], axis=0)
    alts_vect.append(arralts_vect)

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-93.99,-93.81, 16.25, 16.1])
#plt.plot(-93.9373 ,16.1825, markersize=6, marker='d', color='red')
ax.coastlines(resolution="10m")
#ax.text(-93.94, 16.118, '10 km')
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
p= ax.pcolormesh(lon_10km_ev100m, lat_10km_ev100m, alts_vect, cmap = 'viridis', transform=ccrs.PlateCarree())
cbar = plt.colorbar(p, shrink=0.4)
cbar.set_label('metros')
#plt.savefig('./../../2_Semestre/Modelos_metereologicos/Avances Proyecto Final/MapAltitud.png', format = 'png', dpi = 200, bbox_inches = 'tight')

### Uso de suelo INEGI

In [ ]:
rug_vect1 = []
k = 0
for i in range (0, 101):
    arrrug_vect1 = []
    for j in range(0,101):
        arrrug_vect1.append(dfRug_INEGI.Indice[k])
        k=k+1
    #Umatrix = np.append(Umatrix, [arrU], axis=0)
    rug_vect1.append(arrrug_vect1)

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-93.99,-93.81, 16.25, 16.1])
#plt.plot(-93.9373 ,16.1825, markersize=6, marker='d', color='red')
ax.coastlines(resolution="10m")
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
p= ax.pcolormesh(lon_10km_ev100m, lat_10km_ev100m, rug_vect1, cmap = 'viridis', transform=ccrs.PlateCarree())
cbar = plt.colorbar(p, shrink=0.4)
cbar.set_label('Índice')

# Lectura de las velocidades cada 1 km (vecino más cercano) Dia con menor Correlación

In [ ]:
df1N_Min = pd.read_csv('../dat/caso_D/df_Min_000102.csv')

In [ ]:
df_N_Min = pd.concat([df1N_Min])

In [ ]:
df_N_Min

In [ ]:
df_N_Min = df_N_Min.drop(['Unnamed: 0'], axis = 1)
df_N_Min = df_N_Min.rename(columns = {'Vel_m/s':'Vels'})
df_N_Min = df_N_Min.drop(['Date', 'Dir'], axis = 1)

In [ ]:
df_N_Min

In [ ]:
df_N_Min = df_N_Min.sort_values(['num', 'Date_UTCMex'])
#df_N = df_N.drop(['num'], axis = 1)
df_N_Min = df_N_Min.set_index('Date_UTCMex')
df_N_Min.index = pd.to_datetime(df_N_Min.index)

In [ ]:
df00_N_Min = df_N_Min.Vels.between_time('00:00', '00:00')

## Indices para los datos de altitud originales (cada 1 km)

Se asignan los 'índices' a los datos de altitud, es decir, se van a extraer los datos de altitud relacionados con los puntos originales, los que se encuentran a cada kilómetro de distancia

In [ ]:
arr1 =np.linspace(0,100,11).tolist()
arr2 =np.linspace(1010, 1110,11).tolist()
arr3 =np.linspace(2020, 2120,11).tolist()
arr4 =np.linspace(3030, 3130,11).tolist()
arr5 =np.linspace(4040, 4140,11).tolist()
arr6 =np.linspace(5050, 5150,11).tolist()
arr7 =np.linspace(6060, 6160,11).tolist()
arr8 =np.linspace(7070, 7170,11).tolist()
arr9 =np.linspace(8080, 8180,11).tolist()
arr10 =np.linspace(9090, 9190,11).tolist()
arr11 =np.linspace(10100, 10200,11).tolist()
arr = arr1 + arr2 + arr3+ arr4+arr5+arr6+arr7+arr8+arr9+arr10+arr11
ind = np.linspace(0,120,121)

In [ ]:
dfAltitudOrig = dfAltitud.iloc[arr]
dfAltitudOrig['index'] = ind
dfAltitudOrig.set_index('index')
dfAltitudp = dfAltitudOrig['Altitudes'].squeeze()
dfAltitudp.index = ind

# Funciones para extrapolación de la velocidad

In [ ]:
#Ley de potencia
def extrapolacion(Uzr, z, zr):
    alpha = (0.37-(0.088*np.log(Uzr)))/(1-(0.088*np.log(zr/10)))
    Uz = Uzr*((z / zr )**(alpha))
    return Uz

In [ ]:
#Ley logaritmica
def extrapolacionlog(Uzr, z, zr, zo):
    Uz = Uzr*((np.log(z/zo))/(np.log(zr/zo)))
    return Uz

# Estructura calculos

### Planteamiento de asignar las velocidades a cada altitud

### El procedimiento consiste en asignar las velocidades extraídas, las de los puntos originales (las velocidades con una resolución de 1 kilómetro). Para cada punto denttro de la nueva malla se le asignaran 4 velocidades, es decir, las 4 esquinas con velocidades extraídas

Con el punto de arriba a la izquierda del cuadrante BIEN

In [ ]:
dfU00_N_1_Min = []
dfAlt1 = []
for x in range(0,11): #Bloques diferentes
    if x == 0:
        if x != 10:
            for z in range(0,10): #Movimiento vertical
                for j in range(0,11): #Hasta donde llega en horizontal ###BIEN
                    if j != 10:
                        for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                            dfU00_N_1_Min.append(df00_N_Min[j+(10*x)]);                              
                            dfAlt1.append(dfAltitudp[j+(10*x)])
                    else:
                        dfU00_N_1_Min.append(df00_N_Min[j+(10*x)]);                   
                        dfAlt1.append(dfAltitudp[j+(10*x)])
        else:
            for j in range(0,11): #Hasta donde llega en horizontal ##BIEN
                if j != 10:
                    for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                        dfU00_N_1_Min.append(df00_N_Min[j+(10*x)]);                            
                        dfAlt1.append(dfAltitudp[j+(10*x)])
                else:
                    dfU00_N_1_Min.append(df00_N_Min[j+(10*x)]);                                     
                    dfAlt1.append(dfAltitudp[j+(10*x)])
    else:
        if x != 10:
            for z in range(0,10): #Movimiento vertical
                for j in range(0,11): #Hasta donde llega en horizontal ###BIEN
                    if j != 10:
                        for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                            dfU00_N_1_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                                
                            dfAlt1.append(dfAltitudp[j+(10*x)+(1*x)])
                    else:
                        dfU00_N_1_Min.append(df00_N_Min[j+(10*x)+(1*x)]);
                        dfAlt1.append(dfAltitudp[j+(10*x)+(1*x)])
        else:
            for j in range(0,11): #Hasta donde llega en horizontal ##BIEN
                if j != 10:
                    for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                        dfU00_N_1_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                          
                        dfAlt1.append(dfAltitudp[j+(10*x)+(1*x)])
                else:
                    dfU00_N_1_Min.append(df00_N_Min[j+(10*x)+(1*x)]);       
                    dfAlt1.append(dfAltitudp[j+(10*x)+(1*x)])

Con el punto arriba a la derecha

In [ ]:
dfU00_N_2_Min = []; 
dfAlt2 = []
for x in range(0,11): #Bloques diferentes
    if x == 0:
        if x != 10:
            for z in range(0,10): #Movimiento vertical
                for j in range(0,11): #Hasta donde llega en horizontal ###BIEN
                    if j != 0:
                        for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                            dfU00_N_2_Min.append(df00_N_Min[j+(10*x)]);                         
                            dfAlt2.append(dfAltitudp[j+(10*x)])
                    else:
                        dfU00_N_2_Min.append(df00_N_Min[j+(10*x)]);                        
                        dfAlt2.append(dfAltitudp[j+(10*x)])
        else:
            for j in range(0,11): #Hasta donde llega en horizontal ##BIEN
                if j != 0:
                    for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                        dfU00_N_2_Min.append(df00_N_Min[j+(10*x)]);                 
                        dfAlt2.append(dfAltitudp[j+(10*x)])
                else:
                    dfU00_N_2_Min.append(df00_N_Min[j+(10*x)]);                     
                    dfAlt2.append(dfAltitudp[j+(10*x)])
    else:
        if x != 10:
            for z in range(0,10): #Movimiento vertical
                for j in range(0,11): #Hasta donde llega en horizontal ###BIEN
                    if j != 0:
                        for i in range (0,10): #Repeticiones seguidas (10 v
                            dfU00_N_2_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                 
                            dfAlt2.append(dfAltitudp[j+(10*x)+(1*x)])
                    else:
                        dfU00_N_2_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                         
                        dfAlt2.append(dfAltitudp[j+(10*x)+(1*x)])
        else:
            for j in range(0,11): #Hasta donde llega en horizontal ##BIEN
                if j != 0:
                    for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                        dfU00_N_2_Min.append(df00_N_Min[j+(10*x)+(1*x)]);             
                        dfAlt2.append(dfAltitudp[j+(10*x)+(1*x)])
                else:
                    dfU00_N_2_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                       
                    dfAlt2.append(dfAltitudp[j+(10*x)+(1*x)])

Abajo a la izquierda

In [ ]:
dfU00_N_3_Min = []; 
dfAlt3 = []
for x in range(0,11): #Bloques diferentes
    if x == 0:
        for j in range(0,11): #Hasta donde llega en horizontal ##BIEN
            if j != 10:
                for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                        dfU00_N_3_Min.append(df00_N_Min[j+(10*x)]);           
                        dfAlt3.append(dfAltitudp[j+(10*x)])
            else:
                    dfU00_N_3_Min.append(df00_N_Min[j+(10*x)]);                         
                    dfAlt3.append(dfAltitudp[j+(10*x)])
    else:
        for z in range(0,10): #Movimiento vertical
            for j in range(0,11): #Hasta donde llega en horizontal ###BIEN
                if j != 10:
                    for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                        dfU00_N_3_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                                 
                        dfAlt3.append(dfAltitudp[j+(10*x)+(1*x)])
                else:
                    dfU00_N_3_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                     
                    dfAlt3.append(dfAltitudp[j+(10*x)+(1*x)])

Abajo a la derecha

In [ ]:
dfU00_N_4_Min = []; 
dfAlt4 = []
for x in range(0,11): #Bloques diferentes
    if x == 0:
        for j in range(0,11): #Hasta donde llega en horizontal ##BIEN
            if j != 0:
                for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                        dfU00_N_4_Min.append(df00_N_Min[j+(10*x)]);                                             
                        dfAlt4.append(dfAltitudp[j+(10*x)])
            else:
                    dfU00_N_4_Min.append(df00_N_Min[j+(10*x)]);                                       
                    dfAlt4.append(dfAltitudp[j+(10*x)])
    else:
        for z in range(0,10): #Movimiento vertical
            for j in range(0,11): #Hasta donde llega en horizontal ###BIEN
                if j != 0:
                    for i in range (0,10): #Repeticiones seguidas (10 veces) ###BIEN
                        dfU00_N_4_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                               
                        dfAlt4.append(dfAltitudp[j+(10*x)+(1*x)])
                else:
                    dfU00_N_4_Min.append(df00_N_Min[j+(10*x)+(1*x)]);                    
                    dfAlt4.append(dfAltitudp[j+(10*x)+(1*x)])

### Se realiza un dataframe para cada una de las horas colocando las 4 velocidades asignadas, así como la diferencía de altitud entre los puntos 'originales' y los que no se conoce la velocidad del viento. Así como la longitud de rugosidad correspondiente a cada punto

In [ ]:
dfV00_N_Min = dfAltitud_1;                                                                      
dfV00_N_Min['Uz_UL'] = np.abs(dfU00_N_1_Min);                                                       
dfV00_N_Min['zr_UL'] = 80-(dfAlt1-dfV00_N_Min.Altitudes);                                           
dfV00_N_Min['Uz_UR'] = np.abs(dfU00_N_2_Min);                                                       
dfV00_N_Min['zr_UR'] = 80-(dfAlt2-dfV00_N_Min.Altitudes);                                           
dfV00_N_Min['Uz_DL'] = np.abs(dfU00_N_3_Min);                                                       
dfV00_N_Min['zr_DL'] = 80-(dfAlt3-dfV00_N_Min.Altitudes);                                           
dfV00_N_Min['Uz_DR'] = np.abs(dfU00_N_4_Min);                                                       
dfV00_N_Min['zr_DR'] = 80-(dfAlt4-dfV00_N_Min.Altitudes);                                           
dfV00_N_Min['zo_1'] = dfRug_INEGI.zo_1;                                                                                                                 
dfV00_N_Min = dfV00_N_Min.rename(columns={'Altitudes':'z1','Longitudes':'lon', 'Latitudes':'lat'}); 
#


### Se realiza la extrapolación de las velocidades de viento.

In [ ]:
U_UL_log_00_N_Min = extrapolacionlog(dfV00_N_Min.Uz_UL, dfV00_N_Min.zr_UL, 80, dfV00_N_Min.zo_1);
U_UR_log_00_N_Min = extrapolacionlog(dfV00_N_Min.Uz_UR, dfV00_N_Min.zr_UR, 80, dfV00_N_Min.zo_1);
U_DL_log_00_N_Min = extrapolacionlog(dfV00_N_Min.Uz_DL, dfV00_N_Min.zr_DL, 80, dfV00_N_Min.zo_1);
U_DR_log_00_N_Min = extrapolacionlog(dfV00_N_Min.Uz_DR, dfV00_N_Min.zr_DR, 80, dfV00_N_Min.zo_1);
U_UL_pot_00_N_Min =    extrapolacion(dfV00_N_Min.Uz_UL, dfV00_N_Min.zr_UL, 80);              
U_UR_pot_00_N_Min =    extrapolacion(dfV00_N_Min.Uz_UR, dfV00_N_Min.zr_UR, 80);              
U_DL_pot_00_N_Min =    extrapolacion(dfV00_N_Min.Uz_DL, dfV00_N_Min.zr_DL, 80);              
U_DR_pot_00_N_Min =    extrapolacion(dfV00_N_Min.Uz_DR, dfV00_N_Min.zr_DR, 80);                  
#
U

### Se establece el nuevo dataframe con las velocidades extrapoladas

In [ ]:
Velo00_N_Min = pd.DataFrame() ;               
Velo00_N_Min['lon'] = dfV00_N_Min.Longitud;            
Velo00_N_Min['lat'] = dfV00_N_Min.Latitud;            
Velo00_N_Min['Vel_UL_log'] = U_UL_log_00_N_Min;   
Velo00_N_Min['Vel_UL_pot'] = U_UL_pot_00_N_Min;   
Velo00_N_Min['Vel_UR_log'] = U_UR_log_00_N_Min;   
Velo00_N_Min['Vel_UR_pot'] = U_UR_pot_00_N_Min;   
Velo00_N_Min['Vel_DL_log'] = U_DL_log_00_N_Min;   
Velo00_N_Min['Vel_DL_pot'] = U_DL_pot_00_N_Min;   
Velo00_N_Min['Vel_DR_log'] = U_DR_log_00_N_Min;   
Velo00_N_Min['Vel_DR_pot'] = U_DR_pot_00_N_Min;   


### Se realiza la limpia de los valores donde queda - infinito y se reemplazan por cero

In [ ]:
Velo00_N_Min = Velo00_N_Min.where(Velo00_N_Min!=-np.inf, 0, axis=0);
Velo00_N_Min = Velo00_N_Min.fillna(0)

## Se realiza el calculo del promedio 

In [ ]:
Vel_4p_pot_00_N_Min = (Velo00_N_Min.Vel_UL_pot + Velo00_N_Min.Vel_UR_pot + Velo00_N_Min.Vel_DL_pot + Velo00_N_Min.Vel_DR_pot)/4;

In [ ]:
Vel_4p_log_00_N_Min = []
for i in range(len(Velo00_N_Min)):
    n=0
    if Velo00_N_Min.Vel_UL_log[i] != 0:
        n = n+1
    if Velo00_N_Min.Vel_UR_log[i] != 0:
        n = n+1 
    if Velo00_N_Min.Vel_DL_log[i] != 0:
        n = n+1 
    if Velo00_N_Min.Vel_DR_log[i] != 0:
        n = n+1
    Vel_log_00_N_Min= (Velo00_N_Min.Vel_UL_log[i] + Velo00_N_Min.Vel_UR_log[i] + Velo00_N_Min.Vel_DL_log[i] + Velo00_N_Min.Vel_DR_log[i])/n
    Vel_4p_log_00_N_Min.append(Vel_log_00_N_Min)

## Formación de los vectores para las gráficas 

In [ ]:
Vel_4p_logm_00_N_Min = [];
k = 0
for i in range (0, 101):
    arrU_4p_1_00_N_Min = [] ;
    
    for j in range(0,101):
        arrU_4p_1_00_N_Min.append(Vel_4p_log_00_N_Min[k]) ;        
        k=k+1
    #Umatrix = np.append(Umatrix, [arrU], axis=0)
    Vel_4p_logm_00_N_Min.append(arrU_4p_1_00_N_Min);

In [ ]:
List_G = list(range(100,-1, -1))

In [ ]:
List_G;

In [ ]:
Vel_4p_logm_00_N_1_Min = []; 
kk = []
for j in range(0,101):
    k = List_G[j]
    #kk.append(k)
    arrU_logm_00_N_1_Min =  Vel_4p_logm_00_N_Min[k] ; 
    Vel_4p_logm_00_N_1_Min.append(arrU_logm_00_N_1_Min);

In [ ]:
vmin = min(Vel_4p_log_00_N_Min)
vmax = max(Vel_4p_log_00_N_Min)
cax = plt.imshow(Vel_4p_logm_00_N_1_Min, extent=(-93.94673, -93.85307, 16.134575, 16.224533), origin='lower', cmap = 'jet', vmin=vmin, vmax=vmax)
plt.scatter(dfcoor_Or.Longitud, dfcoor_Or.Latitud, c='k', marker='.')
cbar=plt.colorbar(cax)
cbar.ax.set_ylabel('m/s', rotation=270)
plt.title('00:00 06/03/2016 80 mts')
plt.xlabel('Longitud [-]')
plt.ylabel('Latitud [-]')
plt.show()

# Interpolación para serie temporal de una día (93.862222 W, 16.213611 N)

In [ ]:
dateMx = pd.date_range('2016-03-06 00:00:00', '2016-03-06 23:00:00', freq='1H')

In [ ]:
dateMx

In [ ]:
df_00_F_Min = pd.DataFrame(Vel_4p_log_00_N_Min, columns = ['Vel'])
df_00_F_Min['Longitud'] = dfcoor.Longitud
df_00_F_Min['Latitud'] = dfcoor.Latitud

df_01_F_Min = pd.DataFrame(Vel_4p_log_01_N_Min, columns = ['Vel'])
df_01_F_Min['Longitud'] = dfcoor.Longitud
df_01_F_Min['Latitud'] = dfcoor.Latitud

df_02_F_Min = pd.DataFrame(Vel_4p_log_02_N_Min, columns = ['Vel'])
df_02_F_Min['Longitud'] = dfcoor.Longitud
df_02_F_Min['Latitud'] = dfcoor.Latitud

df_03_F_Min = pd.DataFrame(Vel_4p_log_03_N_Min, columns = ['Vel'])
df_03_F_Min['Longitud'] = dfcoor.Longitud
df_03_F_Min['Latitud'] = dfcoor.Latitud

df_04_F_Min = pd.DataFrame(Vel_4p_log_04_N_Min, columns = ['Vel'])
df_04_F_Min['Longitud'] = dfcoor.Longitud
df_04_F_Min['Latitud'] = dfcoor.Latitud

df_05_F_Min = pd.DataFrame(Vel_4p_log_05_N_Min, columns = ['Vel'])
df_05_F_Min['Longitud'] = dfcoor.Longitud
df_05_F_Min['Latitud'] = dfcoor.Latitud

df_06_F_Min = pd.DataFrame(Vel_4p_log_06_N_Min, columns = ['Vel'])
df_06_F_Min['Longitud'] = dfcoor.Longitud
df_06_F_Min['Latitud'] = dfcoor.Latitud

df_07_F_Min = pd.DataFrame(Vel_4p_log_07_N_Min, columns = ['Vel'])
df_07_F_Min['Longitud'] = dfcoor.Longitud
df_07_F_Min['Latitud'] = dfcoor.Latitud

df_08_F_Min = pd.DataFrame(Vel_4p_log_08_N_Min, columns = ['Vel'])
df_08_F_Min['Longitud'] = dfcoor.Longitud
df_08_F_Min['Latitud'] = dfcoor.Latitud

df_09_F_Min = pd.DataFrame(Vel_4p_log_09_N_Min, columns = ['Vel'])
df_09_F_Min['Longitud'] = dfcoor.Longitud
df_09_F_Min['Latitud'] = dfcoor.Latitud

df_10_F_Min = pd.DataFrame(Vel_4p_log_10_N_Min, columns = ['Vel'])
df_10_F_Min['Longitud'] = dfcoor.Longitud
df_10_F_Min['Latitud'] = dfcoor.Latitud

df_11_F_Min = pd.DataFrame(Vel_4p_log_11_N_Min, columns = ['Vel'])
df_11_F_Min['Longitud'] = dfcoor.Longitud
df_11_F_Min['Latitud'] = dfcoor.Latitud

df_12_F_Min = pd.DataFrame(Vel_4p_log_12_N_Min, columns = ['Vel'])
df_12_F_Min['Longitud'] = dfcoor.Longitud
df_12_F_Min['Latitud'] = dfcoor.Latitud

df_13_F_Min = pd.DataFrame(Vel_4p_log_13_N_Min, columns = ['Vel'])
df_13_F_Min['Longitud'] = dfcoor.Longitud
df_13_F_Min['Latitud'] = dfcoor.Latitud

df_14_F_Min = pd.DataFrame(Vel_4p_log_14_N_Min, columns = ['Vel'])
df_14_F_Min['Longitud'] = dfcoor.Longitud
df_14_F_Min['Latitud'] = dfcoor.Latitud

df_15_F_Min = pd.DataFrame(Vel_4p_log_15_N_Min, columns = ['Vel'])
df_15_F_Min['Longitud'] = dfcoor.Longitud
df_15_F_Min['Latitud'] = dfcoor.Latitud

df_16_F_Min = pd.DataFrame(Vel_4p_log_16_N_Min, columns = ['Vel'])
df_16_F_Min['Longitud'] = dfcoor.Longitud
df_16_F_Min['Latitud'] = dfcoor.Latitud

df_17_F_Min = pd.DataFrame(Vel_4p_log_17_N_Min, columns = ['Vel'])
df_17_F_Min['Longitud'] = dfcoor.Longitud
df_17_F_Min['Latitud'] = dfcoor.Latitud

df_18_F_Min = pd.DataFrame(Vel_4p_log_18_N_Min, columns = ['Vel'])
df_18_F_Min['Longitud'] = dfcoor.Longitud
df_18_F_Min['Latitud'] = dfcoor.Latitud

df_19_F_Min = pd.DataFrame(Vel_4p_log_19_N_Min, columns = ['Vel'])
df_19_F_Min['Longitud'] = dfcoor.Longitud
df_19_F_Min['Latitud'] = dfcoor.Latitud

df_20_F_Min = pd.DataFrame(Vel_4p_log_20_N_Min, columns = ['Vel'])
df_20_F_Min['Longitud'] = dfcoor.Longitud
df_20_F_Min['Latitud'] = dfcoor.Latitud

df_21_F_Min = pd.DataFrame(Vel_4p_log_21_N_Min, columns = ['Vel'])
df_21_F_Min['Longitud'] = dfcoor.Longitud
df_21_F_Min['Latitud'] = dfcoor.Latitud

df_22_F_Min = pd.DataFrame(Vel_4p_log_22_N_Min, columns = ['Vel'])
df_22_F_Min['Longitud'] = dfcoor.Longitud
df_22_F_Min['Latitud'] = dfcoor.Latitud

df_23_F_Min = pd.DataFrame(Vel_4p_log_23_N_Min, columns = ['Vel'])
df_23_F_Min['Longitud'] = dfcoor.Longitud
df_23_F_Min['Latitud'] = dfcoor.Latitud

In [ ]:
df_00_Min= df_00_F_Min.loc[df_00_F_Min['Longitud'] < -93.9372] ; df_12_Min= df_12_F_Min.loc[df_12_F_Min['Longitud'] < -93.9372]
df_00_Min= df_00_Min.loc[df_00_Min['Longitud'] > -93.9375]     ; df_12_Min= df_12_Min.loc[df_12_Min['Longitud'] > -93.9375]
df_00_Min= df_00_Min.loc[df_00_Min['Latitud'] < 16.183]        ; df_12_Min= df_12_Min.loc[df_12_Min['Latitud'] < 16.183]
df_00_Min= df_00_Min.loc[df_00_Min['Latitud'] > 16.1820]       ; df_12_Min= df_12_Min.loc[df_12_Min['Latitud'] > 16.1820]

df_01_Min= df_01_F_Min.loc[df_01_F_Min['Longitud'] < -93.9372] ; df_13_Min= df_13_F_Min.loc[df_13_F_Min['Longitud'] < -93.9372]
df_01_Min= df_01_Min.loc[df_01_Min['Longitud'] > -93.9375]     ; df_13_Min= df_13_Min.loc[df_13_Min['Longitud'] > -93.9375]
df_01_Min= df_01_Min.loc[df_01_Min['Latitud'] < 16.183]        ; df_13_Min= df_13_Min.loc[df_13_Min['Latitud'] < 16.183]
df_01_Min= df_01_Min.loc[df_01_Min['Latitud'] > 16.1820]       ; df_13_Min= df_13_Min.loc[df_13_Min['Latitud'] > 16.1820]

df_02_Min= df_02_F_Min.loc[df_02_F_Min['Longitud'] < -93.9372] ; df_14_Min= df_14_F_Min.loc[df_14_F_Min['Longitud'] < -93.9372]
df_02_Min= df_02_Min.loc[df_02_Min['Longitud'] > -93.9375]     ; df_14_Min= df_14_Min.loc[df_14_Min['Longitud'] > -93.9375]
df_02_Min= df_02_Min.loc[df_02_Min['Latitud'] < 16.183]        ; df_14_Min= df_14_Min.loc[df_14_Min['Latitud'] < 16.183]
df_02_Min= df_02_Min.loc[df_02_Min['Latitud'] > 16.1820]       ; df_14_Min= df_14_Min.loc[df_14_Min['Latitud'] > 16.1820]

df_03_Min= df_03_F_Min.loc[df_03_F_Min['Longitud'] < -93.9372] ; df_15_Min= df_15_F_Min.loc[df_15_F_Min['Longitud'] < -93.9372]
df_03_Min= df_03_Min.loc[df_03_Min['Longitud'] > -93.9375]     ; df_15_Min= df_15_Min.loc[df_15_Min['Longitud'] > -93.9375]
df_03_Min= df_03_Min.loc[df_03_Min['Latitud'] < 16.183]        ; df_15_Min= df_15_Min.loc[df_15_Min['Latitud'] < 16.183]
df_03_Min= df_03_Min.loc[df_03_Min['Latitud'] > 16.1820]       ; df_15_Min= df_15_Min.loc[df_15_Min['Latitud'] > 16.1820]

df_04_Min= df_04_F_Min.loc[df_04_F_Min['Longitud'] < -93.9372] ; df_16_Min= df_16_F_Min.loc[df_16_F_Min['Longitud'] < -93.9372]
df_04_Min= df_04_Min.loc[df_04_Min['Longitud'] > -93.9375]     ; df_16_Min= df_16_Min.loc[df_16_Min['Longitud'] > -93.9375]
df_04_Min= df_04_Min.loc[df_04_Min['Latitud'] < 16.183]        ; df_16_Min= df_16_Min.loc[df_16_Min['Latitud'] < 16.183]
df_04_Min= df_04_Min.loc[df_04_Min['Latitud'] > 16.1820]       ; df_16_Min= df_16_Min.loc[df_16_Min['Latitud'] > 16.1820]

df_05_Min= df_05_F_Min.loc[df_05_F_Min['Longitud'] < -93.9372] ; df_17_Min= df_17_F_Min.loc[df_17_F_Min['Longitud'] < -93.9372]
df_05_Min= df_05_Min.loc[df_05_Min['Longitud'] > -93.9375]     ; df_17_Min= df_17_Min.loc[df_17_Min['Longitud'] > -93.9375]
df_05_Min= df_05_Min.loc[df_05_Min['Latitud'] < 16.183]        ; df_17_Min= df_17_Min.loc[df_17_Min['Latitud'] < 16.183]
df_05_Min= df_05_Min.loc[df_05_Min['Latitud'] > 16.1820]       ; df_17_Min= df_17_Min.loc[df_17_Min['Latitud'] > 16.1820]

df_06_Min= df_06_F_Min.loc[df_06_F_Min['Longitud'] < -93.9372] ; df_18_Min= df_18_F_Min.loc[df_18_F_Min['Longitud'] < -93.9372]
df_06_Min= df_06_Min.loc[df_06_Min['Longitud'] > -93.9375]     ; df_18_Min= df_18_Min.loc[df_18_Min['Longitud'] > -93.9375]
df_06_Min= df_06_Min.loc[df_06_Min['Latitud'] < 16.183]        ; df_18_Min= df_18_Min.loc[df_18_Min['Latitud'] < 16.183]
df_06_Min= df_06_Min.loc[df_06_Min['Latitud'] > 16.1820]       ; df_18_Min= df_18_Min.loc[df_18_Min['Latitud'] > 16.1820]

df_07_Min= df_07_F_Min.loc[df_07_F_Min['Longitud'] < -93.9372] ; df_19_Min= df_19_F_Min.loc[df_19_F_Min['Longitud'] < -93.9372]
df_07_Min= df_07_Min.loc[df_07_Min['Longitud'] > -93.9375]     ; df_19_Min= df_19_Min.loc[df_19_Min['Longitud'] > -93.9375]
df_07_Min= df_07_Min.loc[df_07_Min['Latitud'] < 16.183]        ; df_19_Min= df_19_Min.loc[df_19_Min['Latitud'] < 16.183]
df_07_Min= df_07_Min.loc[df_07_Min['Latitud'] > 16.1820]       ; df_19_Min= df_19_Min.loc[df_19_Min['Latitud'] > 16.1820]

df_08_Min= df_08_F_Min.loc[df_08_F_Min['Longitud'] < -93.9372] ; df_20_Min= df_20_F_Min.loc[df_20_F_Min['Longitud'] < -93.9372]
df_08_Min= df_08_Min.loc[df_08_Min['Longitud'] > -93.9375]     ; df_20_Min= df_20_Min.loc[df_20_Min['Longitud'] > -93.9375]
df_08_Min= df_08_Min.loc[df_08_Min['Latitud'] < 16.183]        ; df_20_Min= df_20_Min.loc[df_20_Min['Latitud'] < 16.183]
df_08_Min= df_08_Min.loc[df_08_Min['Latitud'] > 16.1820]       ; df_20_Min= df_20_Min.loc[df_20_Min['Latitud'] > 16.1820]

df_09_Min= df_09_F_Min.loc[df_09_F_Min['Longitud'] < -93.9372] ; df_21_Min= df_21_F_Min.loc[df_21_F_Min['Longitud'] < -93.9372]
df_09_Min= df_09_Min.loc[df_09_Min['Longitud'] > -93.9375]     ; df_21_Min= df_21_Min.loc[df_21_Min['Longitud'] > -93.9375]
df_09_Min= df_09_Min.loc[df_09_Min['Latitud'] < 16.183]        ; df_21_Min= df_21_Min.loc[df_21_Min['Latitud'] < 16.183]
df_09_Min= df_09_Min.loc[df_09_Min['Latitud'] > 16.1820]       ; df_21_Min= df_21_Min.loc[df_21_Min['Latitud'] > 16.1820]

df_10_Min= df_10_F_Min.loc[df_10_F_Min['Longitud'] < -93.9372] ; df_22_Min= df_22_F_Min.loc[df_22_F_Min['Longitud'] < -93.9372]
df_10_Min= df_10_Min.loc[df_10_Min['Longitud'] > -93.9375]     ; df_22_Min= df_22_Min.loc[df_22_Min['Longitud'] > -93.9375]
df_10_Min= df_10_Min.loc[df_10_Min['Latitud'] < 16.183]        ; df_22_Min= df_22_Min.loc[df_22_Min['Latitud'] < 16.183]
df_10_Min= df_10_Min.loc[df_10_Min['Latitud'] > 16.1820]       ; df_22_Min= df_22_Min.loc[df_22_Min['Latitud'] > 16.1820]

df_11_Min= df_11_F_Min.loc[df_11_F_Min['Longitud'] < -93.9372] ; df_23_Min= df_23_F_Min.loc[df_23_F_Min['Longitud'] < -93.9372]
df_11_Min= df_11_Min.loc[df_11_Min['Longitud'] > -93.9375]     ; df_23_Min= df_23_Min.loc[df_23_Min['Longitud'] > -93.9375]
df_11_Min= df_11_Min.loc[df_11_Min['Latitud'] < 16.183]        ; df_23_Min= df_23_Min.loc[df_23_Min['Latitud'] < 16.183]
df_11_Min= df_11_Min.loc[df_11_Min['Latitud'] > 16.1820]       ; df_23_Min= df_23_Min.loc[df_23_Min['Latitud'] > 16.1820]

In [ ]:
df_18_Min

In [ ]:
ST_WRF_Min = pd.concat([df_00_Min, df_01_Min, df_02_Min, df_03_Min, df_04_Min, df_05_Min, df_06_Min, df_07_Min,
                    df_08_Min, df_09_Min, df_10_Min, df_11_Min, df_12_Min, df_13_Min, df_14_Min, df_15_Min,
                    df_16_Min, df_17_Min, df_18_Min, df_19_Min, df_20_Min, df_21_Min, df_22_Min, df_23_Min])

In [ ]:
ST_WRF_Min['DatMX'] = dateMx
ST_WRF_Min = ST_WRF_Min.set_index('DatMX')
ST_WRF_Min.index = pd.to_datetime(ST_WRF_Min.index)

## Datos WRF IB

In [ ]:
datWRF_min = pd.read_csv('datST_IB.csv')
datWRF_min = datWRF_min.set_index('Date_UTCMex')
datWRF_min.index = pd.to_datetime(datWRF_min.index);

In [ ]:
datWRF_min

In [ ]:
datWRF09 = datWRF_min.loc['2016-03-06':'2016-03-06'] #YYYY-MM-DD
#datReal03Vel = datReal03.WTG01_AmbientWinSpeedAvg.between_time('00:00', '23:50').to_frame()
#datReal03Vel = datReal03Vel.rename(columns={'WTG01_AmbientWindSpeedAvg':'Vel'})
#datReal03Vel = datReal03Vel.resample('H').mean()

In [ ]:
datWRF09

# Datos WRF Nearest

In [ ]:
datWRF_min_Near = pd.read_csv('datST_near_A.csv')
datWRF_min_Near = datWRF_min_Near.set_index('Date_UTCMex')
datWRF_min_Near.index = pd.to_datetime(datWRF_min_Near.index);

In [ ]:
datWRF09_Near = datWRF_min_Near.loc['2016-03-06':'2016-03-06']

# Datos Reales

In [ ]:
datReal_min = pd.read_csv('Datos_WTG_Arriaga_2016.csv')
datReal_min = datReal_min.set_index('PCTimeStamp')
datReal_min.index = pd.to_datetime(datReal_min.index);

In [ ]:
datReal09 = datReal_min.loc['2016-03-06':'2016-03-06'] #YYYY-MM-DD
datReal09Vel = datReal09.WTG01_AmbientWindSpeedAvg.between_time('00:00', '23:50').to_frame()
datReal09Vel = datReal09Vel.rename(columns={'WTG01_AmbientWindSpeedAvg':'Vel'})
datReal09Vel = datReal09Vel.resample('H').mean()

In [ ]:
datReal09Vel

## Comparación

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.plot(ST_WRF_Min.index, ST_WRF_Min.Vel, color='tab:cyan', label = 'Aumento')
plt.plot(datReal09.index, datReal09.WS_80, color='tab:green', label = 'Real')
plt.plot(datWRF09_Near.index, datWRF09_Near['Vel_m/s'], color='tab:blue', label = 'WRF-VMC')
plt.plot(datWRF09.index, datWRF09['Vel_m/s'], color='tab:red', label = 'WRF-IB')
plt.gca().set(title= 'Velocidad de Viento a 80 m [24.053056 N, 110.585556 W]', xlabel= 'Día Hora', ylabel='[m/s]')
plt.legend()
plt.show()

## Métricas

reales vs IB

In [ ]:
scipy.stats.pearsonr(datReal09Vel.Vel, datWRF09['Vel_m/s'])

In [ ]:
scipy.stats.spearmanr(datReal09Vel.Vel, datWRF09['Vel_m/s'])

In [ ]:
sqrt (mean_squared_error ( datReal09Vel.Vel, datWRF09['Vel_m/s'] ))

reales vs vmc

In [ ]:
scipy.stats.pearsonr(datReal09Vel.Vel, datWRF09_Near['Vel_m/s'])

In [ ]:
scipy.stats.spearmanr(datReal09Vel.Vel, datWRF09_Near['Vel_m/s'])

In [ ]:
sqrt (mean_squared_error ( datReal09Vel.Vel, datWRF09_Near['Vel_m/s'] ))

reales vs aumento

In [ ]:
scipy.stats.pearsonr(datReal09Vel.Vel, ST_WRF_Min.Vel)

In [ ]:
scipy.stats.spearmanr(datReal09Vel.Vel, ST_WRF_Min.Vel)

In [ ]:
sqrt (mean_squared_error ( datReal09Vel.Vel, ST_WRF_Min.Vel))

## Calculo de factor de planta

In [ ]:
#función para determinar la energía generada.
def ENEGEN(Vel):
    if Vel < 3:
        pot = 0 
    if Vel >= 3:
        if Vel < 3.5:
            pot = 10
    if Vel >= 3.5:
        if Vel < 4:
            pot = 20       
    if Vel >= 4:
        if Vel < 4.5:
            pot = 46 
    if Vel >= 4.5:
        if Vel < 5:
            pot = 110
    if Vel >= 5:
        if Vel < 5.5:
            pot = 170 
    if Vel >= 5.5:
        if Vel < 6:
            pot = 240       
    if Vel >= 6:
        if Vel < 6.5:
            pot = 355 
    if Vel >= 6.5:
        if Vel < 7:
            pot = 460      
    if Vel >= 7:
        if Vel < 7.5:
            pot = 580 
    if Vel >= 7.5:
        if Vel < 8:
            pot = 732     
    if Vel >= 8:
        if Vel < 8.5:
            pot = 884 
    if Vel >= 8.5:
        if Vel < 9:
            pot = 1065    
    if Vel >= 9:
        if Vel < 9.5:
            pot = 1245 
    if Vel >= 9.5:
        if Vel < 10:
            pot = 1428
    if Vel >= 10:
        if Vel < 10.5:
            pot = 1612 
    if Vel >= 10.5:
        if Vel < 11:
            pot = 1756
    if Vel >= 11:
        if Vel < 11.5:
            pot = 1900 
    if Vel >= 11.5:
        if Vel < 12:
            pot = 1940
    if Vel >= 12:
        if Vel < 12.5:
            pot = 1968 
    if Vel >= 12.5:
        if Vel < 13:
            pot = 1980
    if Vel >= 13:
        if Vel < 13.5:
            pot = 1990        
    if Vel >= 13.5:
        pot = 2000
    if Vel >25:
        pot = 0
    return(pot)

In [ ]:
En_nominal = 2000*24

In [ ]:
En_real_1 = 0
for i in range(0,24):
    pot = ENEGEN(datReal09.WS_80[i])
    En_real_1 = pot + En_real_1

In [ ]:
En_aum_1 = 0
for i in range(0,24):
    pot = ENEGEN(ST_WRF_Min.Vel[i])
    En_aum_1 = pot + En_aum_1

In [ ]:
En_near_1 = 0
for i in range(0,24):
    pot = ENEGEN(datWRF09_Near['Vel_m/s'][i])
    En_near_1 = pot + En_near_1

In [ ]:
En_IB_1 = 0
for i in range(0,24):
    pot = ENEGEN(datWRF09['Vel_m/s'][i])
    En_IB_1 = pot + En_IB_1

In [ ]:
FP_real_1 = En_real_1/En_nominal
FP_aum_1 = En_aum_1/En_nominal
FP_near_1 = En_near_1/En_nominal
FP_IB_1 = En_IB_1/En_nominal
print(FP_real_1, FP_aum_1, FP_near_1, FP_IB_1)